In [22]:
from pathlib import Path
import napari
import pandas as pd
from cellpose import models, core, io
import pyclesperanto_prototype as cle
from tifffile import imread 
from skimage.measure import regionprops_table
from utils import list_images, read_image, extract_scaling_metadata, segment_organoids_from_cp_labels, extract_organoid_stats_and_merge

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose models
model = models.CellposeModel(gpu=True, model_type="cyto3") 

creating new log file
2026-02-04 11:17:32,492 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2026-02-04 11:17:32,492 [INFO] 
cellpose version: 	3.1.1.3 
platform:       	win32 
python version: 	3.10.19 
torch version:  	2.5.0
2026-02-04 11:17:32,494 [INFO] ** TORCH CUDA version installed and working. **
2026-02-04 11:17:32,495 [INFO] >> cyto3 << model set to be used
2026-02-04 11:17:32,497 [INFO] ** TORCH CUDA version installed and working. **
2026-02-04 11:17:32,497 [INFO] >>>> using GPU (CUDA)
2026-02-04 11:17:32,677 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cyto3
2026-02-04 11:17:32,893 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [23]:
# Define the markers you wish to analyze and its cellular compartment (i.e. cell or membrane)
# ("channel_name", position, location)
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane"), ("Occludin_RFP", 0, "cell")]

In [24]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\F2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\G2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\P

In [25]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id from filename
well_id = filename.split("_")[0]

# Extract x,y,z scaling from .nd2 file metadata in order feed the Z pixel size / XY pixel size ratio into Cellpose
pixel_size_x, pixel_size_y, voxel_size_z = extract_scaling_metadata(image)

assert pixel_size_x == pixel_size_y
z_to_xy_ratio = voxel_size_z / pixel_size_x
print(z_to_xy_ratio)


Image analyzed: B2
Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm
3.017451031446116


In [26]:
# Type down the multiposition index you want to explore
position = 0

print(f"Analyzing multiposition index {position}")
# Generate name for Cellpose prediction for current well and position
cellpose_filename = f"{well_id}_{position}"

# Check if Cellpose prediction is ready, if that is the case load it for next steps
# Construct path to store/check for cytoplasm labels
cellpose_prediction_path = directory_path / "cellpose_labels" / f"{cellpose_filename}.tif"

# Open one of the multipositions in the img file
single_img = img[position]
# Input shape for next step has to be C, Z, Y, X
single_img = single_img.transpose(1, 0, 2, 3)

# Check if the prediction has already been generated and load it
if cellpose_prediction_path.exists():
    print(f"Predictions already calculated for: {cellpose_filename}")
    cytoplasm_labels = imread(cellpose_prediction_path)

else: # cytoplasm labels prediction file not present

    # Keep only 3 meaningful input fluorescence channels to mimick Cellpose GUI normalization
    # 0 membrane marker, 2 nuclei, 3 Cellmask (Numpy 0-index)
    cellpose_input = single_img[[0, 2, 3],:,:,:] # it becomes 0 membrane marker, 1 nuclei, 2 Cellmask (Numpy 0-index)

    # Segment cells with Cellpose using Cellmask (3) and DAPI (2) channel as inputs 
    # (corrected for anisotropy and cell diameter)

    cytoplasm_labels, _, _ = model.eval(
        cellpose_input,                # numpy array
        channels=[3, 2],               # channels=[cyto_chan, nuclear_chan] # Cellmask (ch3, index 2), nuclei (ch2, index 1)
        diameter=20,                   # in pixels (XY), checked with GUI
        do_3D=True,
        anisotropy=z_to_xy_ratio,      # Z pixel size / XY pixel size
        normalize=True,
        flow_threshold=0.4,
        cellprob_threshold=0.0,
        min_size=15,
    )
    del cellpose_input

# Initialize Napari Viewer
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(single_img)
viewer.add_labels(cytoplasm_labels)

Analyzing multiposition index 0
Predictions already calculated for: B2_0


<Labels layer 'cytoplasm_labels' at 0x1f6262ad030>

In [27]:
# Create a dictionary containing all image descriptors
#TODO: Add multiposition index during BP
descriptor_dict = {
            "filename": filename,
            "well_id": well_id,
            }

props_list = []

membrane_labels = None # Variable to check in order to compute membrane_labels just once, avoid repeated GPU ops

# Loop through markers and extract 
for marker_name, ch_nr, location in markers:
    print(f"Analyzing channel: {marker_name} in {location} ...")

    # Region cell or region membrane, generate membrane and extract info from that location if needed
    if location == "cell":
        props = regionprops_table(label_image=cytoplasm_labels,
                                intensity_image=single_img[ch_nr],
                                properties=[
                                    "label",
                                    "area",
                                    "intensity_mean",
                                    "intensity_min",
                                    "intensity_max",
                                    "intensity_std",
                                ],
                            )
        
    elif location == "membrane":
        layer_name = "membrane_labels"

        # Check if layer already exists in the viewer
        if layer_name not in viewer.layers:
            # Generate membrane if needed
            if membrane_labels is None:
                membrane_labels = cle.reduce_labels_to_label_edges(cytoplasm_labels)
                membrane_labels = cle.pull(membrane_labels)

            viewer.add_labels(
                membrane_labels,
                name=layer_name
            )
        else:
            # Layer already present → do nothing
            pass

        props = regionprops_table(label_image=membrane_labels,
                                intensity_image=single_img[ch_nr],
                                properties=[
                                    "label",
                                    "area",
                                    "intensity_mean",
                                    "intensity_min",
                                    "intensity_max",
                                    "intensity_std",
                                ],
                            )
    
    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename intensity_mean column to indicate the specific image
    prefix = f"{location}_{marker_name}"

    rename_map = {
        "area": f"{location}_area",
        "intensity_mean": f"{prefix}_mean_int", # concentration proxy
        "intensity_min":  f"{prefix}_min_int",
        "intensity_max":  f"{prefix}_max_int",
        "intensity_std":  f"{prefix}_std_int",
    }

    props_df.rename(columns=rename_map, inplace=True)

    # Max / mean ratio (puncta vs diffuse signal)
    props_df[f"{prefix}_max_mean_ratio"] = (props_df[f"{prefix}_max_int"] /props_df[f"{prefix}_mean_int"])
    # Total marker content per cell
    props_df[f"{prefix}_sum_int"] = (props_df[f"{prefix}_mean_int"] * props_df[f"{location}_area"])

    # Append each props_df to props_list
    props_list.append(props_df)

# Initialize the df with the first df in the list
props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0    
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

# Obtain rough single organoid outlines by fusing Cellpose labels and dilation, merging, erosion morphological operations
mip_labels, organoid_labels = segment_organoids_from_cp_labels(cytoplasm_labels)
viewer.add_labels(organoid_labels)

# Display final stats (including per cell and per organoid)
final_df = extract_organoid_stats_and_merge(mip_labels, organoid_labels, props_df)
final_df

Analyzing channel: Occludin_RFP in membrane ...
Analyzing channel: Claudin_FITC in membrane ...
Analyzing channel: Occludin_RFP in cell ...
Cells mapped to no organoid: 268 - 17.22% of total cells (1556)


,filename,well_id,organoid,label,membrane_area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,...,cell_Occludin_RFP_min_int,cell_Occludin_RFP_max_int,cell_Occludin_RFP_std_int,cell_Occludin_RFP_max_mean_ratio,cell_Occludin_RFP_sum_int,organoid_area,organoid_perimeter,organoid_eccentricity,organoid_solidity,organoid_extent
0,B2,B2,0,1,211.0,149.018957,108.0,234.0,21.120050,1.570270,...,108.0,240.0,21.544746,1.573534,66500.0,NaN,NaN,NaN,NaN,NaN
1,B2,B2,0,2,485.0,165.012371,108.0,268.0,26.974976,1.624121,...,108.0,317.0,36.163354,1.728295,299888.0,NaN,NaN,NaN,NaN,NaN
2,B2,B2,1,3,580.0,164.425862,101.0,277.0,24.150670,1.684650,...,101.0,302.0,25.163420,1.795133,254536.0,6643.0,326.007143,0.462363,0.951174,0.793668
3,B2,B2,0,4,555.0,146.093694,109.0,201.0,15.278267,1.375829,...,103.0,226.0,16.947238,1.501463,277107.0,NaN,NaN,NaN,NaN,NaN
4,B2,B2,0,5,205.0,136.770732,99.0,180.0,14.682364,1.316071,...,99.0,191.0,15.180623,1.379357,63004.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,B2,B2,0,1552,144.0,190.027778,140.0,271.0,23.898787,1.426107,...,140.0,271.0,23.632413,1.439484,31628.0,NaN,NaN,NaN,NaN,NaN
1552,B2,B2,0,1553,72.0,191.972222,148.0,260.0,25.379876,1.354363,...,146.0,260.0,26.030610,1.370092,15561.0,NaN,NaN,NaN,NaN,NaN
1553,B2,B2,0,1554,44.0,210.727273,145.0,346.0,36.005911,1.641933,...,145.0,346.0,34.434746,1.629229,11468.0,NaN,NaN,NaN,NaN,NaN
1554,B2,B2,19,1555,60.0,203.683333,153.0,269.0,23.551710,1.320678,...,153.0,269.0,23.505524,1.319397,17126.0,21584.0,611.872150,0.209906,0.956865,0.774036
